In [1]:
import numpy as np #importing numpy
import pandas as pd #importing pandas
import PIL #importing pillow for image processing
from PIL import Image#importing image from pillow library
import skimage#importing skimage library
import os#importing os library
import time#importing time library

#Capture start time of the program
start_time = time.time()

In [2]:
from string import digits
#code to remove numbers in a string and assigning it to the variable
r_digits = str.maketrans('', '', digits)
#taking source data folder path to a varible
rawdata_path = 'D://DKIT//dissertation//Dataset_of_Tomato_Leaves//plantvillage//Preprocessed_data//32//32_no_bias//'
#changing current librbay to source data
os.chdir(rawdata_path)
#taking sub-directories to a list
dirs = os.listdir()
directories = []
diseases = []
#Taking directories into a list and extract disease name from folder name
for x in dirs:
    res = x.translate(r_digits).lower()
    diseases.append(res)
    temp = rawdata_path + x + '//'
    directories.append(temp)

In [3]:
for i in directories:
    os.chdir(i)
    pics = os.listdir()
    #reading pics one by one in each directory
    for j in pics:
        image = Image.open(j,mode='r')
        size = (32,32)
        image = image.resize(size)
        image = image.save(j)

In [4]:
#creating different dataframes for saving data in different vector forms
#gray data
gray_df = pd.DataFrame(columns=['mean','disease'])
#rgb data
rgb_df = pd.DataFrame(columns=['mean','disease'])
#red data
red_df = pd.DataFrame(columns=['mean','disease'])
#green data
green_df = pd.DataFrame(columns=['mean','disease'])
#blue data
blue_df = pd.DataFrame(columns=['mean','disease'])

In [5]:
#extracting data from images
x = 0
#creating enpty directories to save the vector from in to a list
gray_test = []
rgb_final = []
red_final = []
green_final = []
blue_final = []
disease = []
#reading data directory by directory
for i in directories:
    os.chdir(i)
    k = diseases[x]
    pics = os.listdir()
    #reading pics one by one in each directory
    for j in pics:
        temp_list = []
        rgb_temp = []
        red_temp = []
        green_temp = []
        blue_temp = []
        gray = []
        data = []
        red = []
        green = []
        blue = []
        #opening the image in rgb format
        image = Image.open(j,mode='r')
        data = image.convert('RGB')
        data = np.array(data)
        #extracting only red components
        red = data[:,:,0]
        #extracting only green components
        green = data[:,:,1]
        #extracting only blue components
        blue = data[:,:,2]
        data = data.ravel()
        #saving rgb vector data into list
        rgb_final.append(data)
        rgb_temp.append(np.mean(data))
        rgb_temp.append(k)
        rgb_df.loc[len(rgb_df)] = rgb_temp
        #saving red channel data into dataframe and vector data to a list.
        red = red.ravel()
        red_final.append(red)
        red_temp.append(np.mean(red))
        red_temp.append(k)
        red_df.loc[len(red_df)] = red_temp
        #saving blue channel data into dataframe and vector data to a list.
        blue = blue.ravel()
        blue_final.append(blue)
        blue_temp.append(np.mean(blue))
        blue_temp.append(k)
        blue_df.loc[len(blue_df)] = blue_temp
        #saving green channel data into dataframe and vector data to a list.
        green = green.ravel()
        green_final.append(green)
        green_temp.append(np.mean(green))
        green_temp.append(k)
        green_df.loc[len(green_df)] = green_temp
        #opening the image in gray format
        gray = image.convert('L')
        gray = np.array(gray)
        gray = gray.ravel()
        gray_test.append(gray)
        temp_list.append(np.mean(gray))
        temp_list.append(k)       
        gray_df.loc[len(gray_df)] = temp_list
    x = x+1

In [6]:
#changing path
os.chdir('D://DKIT//dissertation//Dataset_of_Tomato_Leaves//plantvillage//Preprocessed_data//32//32_no_bias_vector//')

In [7]:
#saving the lists as numpy arrays in the local system
np.save('gray_test',gray_test)
np.save('rgb_data',rgb_final)
np.save('red_data',red_final)
np.save('green_data',green_final)
np.save('blue_data',blue_final)

In [8]:
#saving the dataframe as csv files in the local system
gray_df.to_csv('gray_data.csv',index=False)
rgb_df.to_csv('rgb_data.csv',index=False)
red_df.to_csv('red_data.csv',index=False)
green_df.to_csv('green_data.csv',index=False)
blue_df.to_csv('blue_data.csv',index=False)

# KNN on rgb data

In [9]:
#reading numpy array data
rgb_test = np.load('rgb_data.npy')
#reading csv file
rgb_df = pd.read_csv('rgb_data.csv')

In [10]:
loaded = []
for i in rgb_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)

In [11]:
#assigning features and target
features = loaded
target = rgb_df['disease']
#splitting the data to 80 percent and 20 percent
from sklearn.model_selection import train_test_split
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=24)
#scaling the data using standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)

In [12]:
#K-Model on the data
from sklearn.neighbors import KNeighborsClassifier
kmodel = KNeighborsClassifier(n_neighbors=75)

In [13]:
#k-model fitting and accuracy
kmodel.fit(Ftrain,Ltrain)
print(kmodel.score(Ftest,Ltest))

0.4543859649122807


In [14]:
#importing classification report and confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

In [15]:
#printing results of the kmodel
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.26      0.78      0.39       115
                 early_blight       1.00      0.02      0.03       120
                      healthy       0.42      0.85      0.56       125
                  late_blight       0.46      0.22      0.30       125
                    leaf_mold       0.75      0.58      0.65       127
           septoria_leaf_spot       0.63      0.32      0.43       117
                  target_spot       0.54      0.35      0.42       115
          tomato_mosaic_virus       0.51      0.44      0.48        54
tomato_yellow_leaf_curl_virus       0.83      0.39      0.53       114
      two-spotted_spider_mite       0.48      0.56      0.52       128

                     accuracy                           0.45      1140
                    macro avg       0.59      0.45      0.43      1140
                 weighted avg       0.59      0.45      0.43      1140

[[ 

# Logistic on rgb data

In [16]:
#importing logistic data
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='lbfgs', max_iter=10000)
model.fit(Ftrain,Ltrain)
print(model.score(Ftest,Ltest))

0.6280701754385964


In [17]:
#classification report
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.60      0.64      0.62       115
                 early_blight       0.45      0.44      0.45       120
                      healthy       0.74      0.63      0.68       125
                  late_blight       0.56      0.60      0.58       125
                    leaf_mold       0.69      0.65      0.67       127
           septoria_leaf_spot       0.54      0.58      0.56       117
                  target_spot       0.61      0.63      0.62       115
          tomato_mosaic_virus       0.69      0.78      0.73        54
tomato_yellow_leaf_curl_virus       0.79      0.78      0.79       114
      two-spotted_spider_mite       0.67      0.62      0.65       128

                     accuracy                           0.63      1140
                    macro avg       0.63      0.64      0.63      1140
                 weighted avg       0.63      0.63      0.63      1140

[[7

# SVM(linear) on RGB data

In [18]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='linear')
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.6991228070175438


In [19]:
#classification report
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.63      0.82      0.71       115
                 early_blight       0.50      0.46      0.48       120
                      healthy       0.85      0.79      0.82       125
                  late_blight       0.67      0.67      0.67       125
                    leaf_mold       0.77      0.75      0.76       127
           septoria_leaf_spot       0.67      0.64      0.66       117
                  target_spot       0.68      0.69      0.68       115
          tomato_mosaic_virus       0.78      0.91      0.84        54
tomato_yellow_leaf_curl_virus       0.82      0.82      0.82       114
      two-spotted_spider_mite       0.68      0.58      0.62       128

                     accuracy                           0.70      1140
                    macro avg       0.70      0.71      0.71      1140
                 weighted avg       0.70      0.70      0.70      1140

[[9

# SVM(poly) on RGB data

In [20]:
svm_poly = SVC(kernel='poly')
svm_poly.fit(Ftrain,Ltrain)
y_pred = svm_poly.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.6850877192982456


In [21]:
#classification report
y_predict = svm_poly.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.49      0.85      0.62       115
                 early_blight       0.80      0.30      0.44       120
                      healthy       0.64      0.96      0.77       125
                  late_blight       0.81      0.51      0.63       125
                    leaf_mold       0.81      0.77      0.79       127
           septoria_leaf_spot       0.75      0.56      0.64       117
                  target_spot       0.61      0.76      0.67       115
          tomato_mosaic_virus       0.78      0.52      0.62        54
tomato_yellow_leaf_curl_virus       0.83      0.82      0.82       114
      two-spotted_spider_mite       0.70      0.71      0.71       128

                     accuracy                           0.69      1140
                    macro avg       0.72      0.68      0.67      1140
                 weighted avg       0.72      0.69      0.68      1140

[[ 

# SVM(RBF) on RGB data

In [22]:
svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(Ftrain,Ltrain)
y_pred = svm_rbf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.7675438596491229


In [23]:
#classification report
y_predict = svm_rbf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.77      0.86      0.81       115
                 early_blight       0.55      0.56      0.55       120
                      healthy       0.92      0.91      0.92       125
                  late_blight       0.76      0.77      0.76       125
                    leaf_mold       0.87      0.78      0.82       127
           septoria_leaf_spot       0.76      0.65      0.70       117
                  target_spot       0.70      0.71      0.71       115
          tomato_mosaic_virus       0.78      0.83      0.80        54
tomato_yellow_leaf_curl_virus       0.83      0.87      0.85       114
      two-spotted_spider_mite       0.74      0.77      0.75       128

                     accuracy                           0.77      1140
                    macro avg       0.77      0.77      0.77      1140
                 weighted avg       0.77      0.77      0.77      1140

[[ 

# SVM(sigmoid) on RGB data

In [24]:
svm_si = SVC(kernel='sigmoid')
svm_si.fit(Ftrain,Ltrain)
y_pred = svm_si.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.47017543859649125


In [25]:
#classification report
y_predict = svm_si.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.42      0.52      0.47       115
                 early_blight       0.38      0.25      0.30       120
                      healthy       0.77      0.68      0.72       125
                  late_blight       0.33      0.50      0.40       125
                    leaf_mold       0.74      0.48      0.58       127
           septoria_leaf_spot       0.23      0.46      0.31       117
                  target_spot       0.48      0.43      0.45       115
          tomato_mosaic_virus       0.68      0.56      0.61        54
tomato_yellow_leaf_curl_virus       0.82      0.48      0.61       114
      two-spotted_spider_mite       0.55      0.38      0.45       128

                     accuracy                           0.47      1140
                    macro avg       0.54      0.47      0.49      1140
                 weighted avg       0.53      0.47      0.48      1140

[[6

# Random Forest on RGB 

In [26]:
from sklearn.ensemble import RandomForestClassifier

In [27]:
classifier_rf = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=5,n_estimators=100, oob_score=True)

In [28]:
classifier_rf.fit(Ftrain, Ltrain)

RandomForestClassifier(max_depth=5, n_jobs=-1, oob_score=True, random_state=42)

In [29]:
classifier_rf.oob_score_

0.5373985523141039

In [30]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.46      0.66      0.54       115
                 early_blight       0.67      0.10      0.17       120
                      healthy       0.64      0.89      0.74       125
                  late_blight       0.66      0.35      0.46       125
                    leaf_mold       0.62      0.69      0.65       127
           septoria_leaf_spot       0.51      0.45      0.48       117
                  target_spot       0.53      0.77      0.63       115
          tomato_mosaic_virus       0.50      0.02      0.04        54
tomato_yellow_leaf_curl_virus       0.52      0.85      0.64       114
      two-spotted_spider_mite       0.53      0.47      0.50       128

                     accuracy                           0.55      1140
                    macro avg       0.56      0.53      0.49      1140
                 weighted avg       0.57      0.55      0.51      1140

[[ 

# Gaussian Naive Bayes on RGB data

In [31]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(Ftrain, Ltrain)

GaussianNB()

In [32]:
y_pred = gnb.predict(Ftest)

In [33]:
from sklearn import metrics
metrics.accuracy_score(Ltest, y_pred)

0.4570175438596491

In [34]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.45      0.69      0.55       115
                 early_blight       0.37      0.32      0.34       120
                      healthy       0.46      0.62      0.53       125
                  late_blight       0.34      0.24      0.28       125
                    leaf_mold       0.74      0.45      0.56       127
           septoria_leaf_spot       0.41      0.41      0.41       117
                  target_spot       0.50      0.44      0.47       115
          tomato_mosaic_virus       0.36      0.69      0.47        54
tomato_yellow_leaf_curl_virus       0.62      0.45      0.52       114
      two-spotted_spider_mite       0.42      0.41      0.42       128

                     accuracy                           0.46      1140
                    macro avg       0.47      0.47      0.45      1140
                 weighted avg       0.47      0.46      0.45      1140

[[7

# Bernouli Naive bayes on RGB data

In [35]:
from sklearn.naive_bayes import BernoulliNB

In [36]:
bnb = BernoulliNB()
bnb.fit(Ftrain, Ltrain)

BernoulliNB()

In [37]:
y_pred = bnb.predict(Ftest)

In [38]:
metrics.accuracy_score(Ltest, y_pred)

0.41578947368421054

In [39]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.37      0.65      0.47       115
                 early_blight       0.35      0.23      0.27       120
                      healthy       0.41      0.57      0.48       125
                  late_blight       0.21      0.11      0.15       125
                    leaf_mold       0.68      0.41      0.51       127
           septoria_leaf_spot       0.32      0.51      0.40       117
                  target_spot       0.53      0.44      0.48       115
          tomato_mosaic_virus       0.37      0.48      0.42        54
tomato_yellow_leaf_curl_virus       0.64      0.42      0.51       114
      two-spotted_spider_mite       0.43      0.39      0.41       128

                     accuracy                           0.42      1140
                    macro avg       0.43      0.42      0.41      1140
                 weighted avg       0.43      0.42      0.41      1140

[[7

# Decision Tree on RGB data

In [40]:
from sklearn.tree import DecisionTreeClassifier
classifier_tree = DecisionTreeClassifier()

In [41]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.4368421052631579

In [42]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.52      0.53      0.52       115
                 early_blight       0.20      0.21      0.20       120
                      healthy       0.64      0.58      0.61       125
                  late_blight       0.42      0.43      0.43       125
                    leaf_mold       0.50      0.43      0.46       127
           septoria_leaf_spot       0.41      0.44      0.42       117
                  target_spot       0.40      0.41      0.41       115
          tomato_mosaic_virus       0.25      0.35      0.29        54
tomato_yellow_leaf_curl_virus       0.59      0.51      0.54       114
      two-spotted_spider_mite       0.40      0.39      0.40       128

                     accuracy                           0.43      1140
                    macro avg       0.43      0.43      0.43      1140
                 weighted avg       0.44      0.43      0.44      1140

[[6

# KNN on grayscale data

In [43]:
#reading numpy array data
gray_test = np.load('gray_test.npy')
#reading csv file
gray_df = pd.read_csv('gray_data.csv')

In [44]:
loaded = []
for i in gray_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)

In [45]:
features = loaded
target = gray_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)

In [46]:
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.38859649122807016

In [47]:
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.28      0.74      0.40       124
                 early_blight       0.50      0.01      0.02       114
                      healthy       0.38      0.71      0.50       131
                  late_blight       0.31      0.23      0.26       124
                    leaf_mold       0.50      0.49      0.49       117
           septoria_leaf_spot       0.46      0.44      0.45       114
                  target_spot       0.47      0.24      0.32       111
          tomato_mosaic_virus       0.38      0.25      0.30        60
tomato_yellow_leaf_curl_virus       0.44      0.12      0.19       121
      two-spotted_spider_mite       0.55      0.52      0.53       124

                     accuracy                           0.39      1140
                    macro avg       0.43      0.38      0.35      1140
                 weighted avg       0.43      0.39      0.35      1140

[[9

# Logistic on grayscale data

In [48]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='lbfgs', max_iter=10000)
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.3140350877192982

In [49]:
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.48      0.41      0.44       124
                 early_blight       0.25      0.30      0.27       114
                      healthy       0.40      0.38      0.39       131
                  late_blight       0.18      0.19      0.19       124
                    leaf_mold       0.37      0.42      0.40       117
           septoria_leaf_spot       0.26      0.27      0.27       114
                  target_spot       0.20      0.18      0.19       111
          tomato_mosaic_virus       0.25      0.22      0.23        60
tomato_yellow_leaf_curl_virus       0.28      0.26      0.27       121
      two-spotted_spider_mite       0.41      0.44      0.43       124

                     accuracy                           0.31      1140
                    macro avg       0.31      0.31      0.31      1140
                 weighted avg       0.32      0.31      0.31      1140

[[5

# SVM(linear) on Grayscale data

In [50]:
clf = SVC(kernel='linear')
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.3570175438596491


In [51]:
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.33      0.44      0.38       124
                 early_blight       0.26      0.35      0.30       114
                      healthy       0.53      0.50      0.52       131
                  late_blight       0.23      0.22      0.22       124
                    leaf_mold       0.45      0.45      0.45       117
           septoria_leaf_spot       0.28      0.26      0.27       114
                  target_spot       0.28      0.26      0.27       111
          tomato_mosaic_virus       0.35      0.28      0.31        60
tomato_yellow_leaf_curl_virus       0.39      0.28      0.33       121
      two-spotted_spider_mite       0.52      0.45      0.48       124

                     accuracy                           0.36      1140
                    macro avg       0.36      0.35      0.35      1140
                 weighted avg       0.36      0.36      0.36      1140

[[5

# SVM(poly) on grayscale data

In [52]:
svm_poly = SVC(kernel='poly')
svm_poly.fit(Ftrain,Ltrain)
y_pred = svm_poly.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.5157894736842106


In [53]:
#classification report
y_predict = svm_poly.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.50      0.68      0.57       124
                 early_blight       0.55      0.21      0.30       114
                      healthy       0.50      0.92      0.65       131
                  late_blight       0.49      0.17      0.25       124
                    leaf_mold       0.47      0.63      0.54       117
           septoria_leaf_spot       0.68      0.44      0.53       114
                  target_spot       0.48      0.60      0.53       111
          tomato_mosaic_virus       0.60      0.05      0.09        60
tomato_yellow_leaf_curl_virus       0.44      0.56      0.49       121
      two-spotted_spider_mite       0.69      0.62      0.66       124

                     accuracy                           0.52      1140
                    macro avg       0.54      0.49      0.46      1140
                 weighted avg       0.53      0.52      0.48      1140

[[ 

# SVM(RBF) on grayscale data

In [54]:
svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(Ftrain,Ltrain)
y_pred = svm_rbf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.6333333333333333


In [55]:
#classification report
y_predict = svm_rbf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.74      0.65      0.70       124
                 early_blight       0.46      0.49      0.47       114
                      healthy       0.70      0.86      0.77       131
                  late_blight       0.63      0.56      0.59       124
                    leaf_mold       0.69      0.65      0.67       117
           septoria_leaf_spot       0.58      0.62      0.60       114
                  target_spot       0.54      0.61      0.58       111
          tomato_mosaic_virus       0.57      0.47      0.51        60
tomato_yellow_leaf_curl_virus       0.65      0.60      0.63       121
      two-spotted_spider_mite       0.73      0.70      0.72       124

                     accuracy                           0.63      1140
                    macro avg       0.63      0.62      0.62      1140
                 weighted avg       0.64      0.63      0.63      1140

[[ 

# SVM(sigmoid) on grayscale data

In [56]:
svm_si = SVC(kernel='sigmoid')
svm_si.fit(Ftrain,Ltrain)
y_pred = svm_si.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.29210526315789476


In [57]:
#classification report
y_predict = svm_si.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.39      0.45      0.42       124
                 early_blight       0.19      0.20      0.20       114
                      healthy       0.43      0.39      0.41       131
                  late_blight       0.20      0.17      0.18       124
                    leaf_mold       0.56      0.27      0.37       117
           septoria_leaf_spot       0.18      0.39      0.24       114
                  target_spot       0.21      0.23      0.22       111
          tomato_mosaic_virus       0.43      0.05      0.09        60
tomato_yellow_leaf_curl_virus       0.31      0.32      0.31       121
      two-spotted_spider_mite       0.43      0.31      0.36       124

                     accuracy                           0.29      1140
                    macro avg       0.33      0.28      0.28      1140
                 weighted avg       0.33      0.29      0.29      1140

[[5

# Random Forest on Grayscale

In [58]:
classifier_rf.fit(Ftrain, Ltrain)

RandomForestClassifier(max_depth=5, n_jobs=-1, oob_score=True, random_state=42)

In [59]:
classifier_rf.oob_score_

0.4536082474226804

In [60]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.45      0.64      0.52       124
                 early_blight       0.38      0.14      0.21       114
                      healthy       0.46      0.80      0.59       131
                  late_blight       0.48      0.27      0.35       124
                    leaf_mold       0.56      0.58      0.57       117
           septoria_leaf_spot       0.55      0.47      0.51       114
                  target_spot       0.42      0.71      0.53       111
          tomato_mosaic_virus       0.00      0.00      0.00        60
tomato_yellow_leaf_curl_virus       0.52      0.35      0.42       121
      two-spotted_spider_mite       0.49      0.52      0.50       124

                     accuracy                           0.48      1140
                    macro avg       0.43      0.45      0.42      1140
                 weighted avg       0.45      0.48      0.44      1140

[[ 

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Gaussian Naive bayes on Grayscale

In [61]:
gnb.fit(Ftrain, Ltrain)

GaussianNB()

In [62]:
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.4087719298245614

In [63]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.47      0.69      0.56       124
                 early_blight       0.38      0.39      0.39       114
                      healthy       0.44      0.63      0.52       131
                  late_blight       0.34      0.20      0.25       124
                    leaf_mold       0.61      0.29      0.39       117
           septoria_leaf_spot       0.37      0.47      0.42       114
                  target_spot       0.50      0.38      0.43       111
          tomato_mosaic_virus       0.29      0.57      0.39        60
tomato_yellow_leaf_curl_virus       0.27      0.12      0.16       121
      two-spotted_spider_mite       0.39      0.40      0.40       124

                     accuracy                           0.41      1140
                    macro avg       0.41      0.41      0.39      1140
                 weighted avg       0.41      0.41      0.39      1140

[[8

# Bernouli Naive bayes on grayscale data

In [64]:
bnb.fit(Ftrain, Ltrain)
y_pred = bnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.3447368421052632

In [65]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.39      0.69      0.50       124
                 early_blight       0.28      0.18      0.22       114
                      healthy       0.40      0.54      0.46       131
                  late_blight       0.15      0.08      0.10       124
                    leaf_mold       0.63      0.31      0.41       117
           septoria_leaf_spot       0.31      0.53      0.39       114
                  target_spot       0.36      0.26      0.30       111
          tomato_mosaic_virus       0.18      0.27      0.22        60
tomato_yellow_leaf_curl_virus       0.17      0.09      0.12       121
      two-spotted_spider_mite       0.44      0.44      0.44       124

                     accuracy                           0.34      1140
                    macro avg       0.33      0.34      0.32      1140
                 weighted avg       0.34      0.34      0.32      1140

[[8

# Decision Tree on Grayscale

In [66]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.33771929824561403

In [67]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.45      0.48      0.47       124
                 early_blight       0.22      0.21      0.21       114
                      healthy       0.69      0.57      0.62       131
                  late_blight       0.33      0.34      0.34       124
                    leaf_mold       0.36      0.40      0.38       117
           septoria_leaf_spot       0.35      0.39      0.37       114
                  target_spot       0.31      0.32      0.31       111
          tomato_mosaic_virus       0.21      0.23      0.22        60
tomato_yellow_leaf_curl_virus       0.30      0.27      0.29       121
      two-spotted_spider_mite       0.35      0.31      0.33       124

                     accuracy                           0.36      1140
                    macro avg       0.36      0.35      0.35      1140
                 weighted avg       0.37      0.36      0.36      1140

[[6

# KNN on Red channel data

In [68]:
#reading numpy array data
red_test = np.load('red_data.npy')
#reading csv file
red_df = pd.read_csv('red_data.csv')

In [69]:
loaded = []
for i in red_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
features = loaded
target = red_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
scaler = StandardScaler()
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)

In [70]:
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.43508771929824563

In [71]:
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.35      0.71      0.47       124
                 early_blight       0.67      0.02      0.03       114
                      healthy       0.39      0.81      0.53       131
                  late_blight       0.37      0.23      0.29       124
                    leaf_mold       0.51      0.47      0.49       117
           septoria_leaf_spot       0.49      0.33      0.40       114
                  target_spot       0.43      0.32      0.37       111
          tomato_mosaic_virus       0.55      0.52      0.53        60
tomato_yellow_leaf_curl_virus       0.45      0.21      0.28       121
      two-spotted_spider_mite       0.55      0.69      0.61       124

                     accuracy                           0.44      1140
                    macro avg       0.48      0.43      0.40      1140
                 weighted avg       0.47      0.44      0.40      1140

[[ 

# Logistic on Red channel data

In [72]:
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.32894736842105265

In [73]:
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.52      0.46      0.49       124
                 early_blight       0.25      0.29      0.27       114
                      healthy       0.40      0.37      0.38       131
                  late_blight       0.17      0.18      0.17       124
                    leaf_mold       0.39      0.44      0.41       117
           septoria_leaf_spot       0.26      0.25      0.25       114
                  target_spot       0.28      0.27      0.27       111
          tomato_mosaic_virus       0.25      0.20      0.22        60
tomato_yellow_leaf_curl_virus       0.31      0.27      0.29       121
      two-spotted_spider_mite       0.43      0.49      0.46       124

                     accuracy                           0.33      1140
                    macro avg       0.32      0.32      0.32      1140
                 weighted avg       0.33      0.33      0.33      1140

[[5

# SVM(linear) on Red channel data

In [74]:
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.37719298245614036


In [75]:
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.36      0.48      0.41       124
                 early_blight       0.26      0.33      0.29       114
                      healthy       0.51      0.49      0.50       131
                  late_blight       0.25      0.25      0.25       124
                    leaf_mold       0.46      0.44      0.45       117
           septoria_leaf_spot       0.28      0.27      0.27       114
                  target_spot       0.28      0.26      0.27       111
          tomato_mosaic_virus       0.43      0.33      0.37        60
tomato_yellow_leaf_curl_virus       0.40      0.31      0.35       121
      two-spotted_spider_mite       0.59      0.56      0.57       124

                     accuracy                           0.38      1140
                    macro avg       0.38      0.37      0.37      1140
                 weighted avg       0.38      0.38      0.38      1140

[[5

# SVM(poly) on Red data

In [76]:
svm_poly.fit(Ftrain,Ltrain)
y_pred = svm_poly.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.5412280701754386


In [77]:
#classification report
y_predict = svm_poly.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.57      0.59      0.58       124
                 early_blight       0.49      0.17      0.25       114
                      healthy       0.47      0.96      0.63       131
                  late_blight       0.63      0.31      0.42       124
                    leaf_mold       0.68      0.59      0.63       117
           septoria_leaf_spot       0.64      0.48      0.55       114
                  target_spot       0.45      0.70      0.55       111
          tomato_mosaic_virus       0.62      0.13      0.22        60
tomato_yellow_leaf_curl_virus       0.46      0.59      0.52       121
      two-spotted_spider_mite       0.70      0.64      0.67       124

                     accuracy                           0.54      1140
                    macro avg       0.57      0.52      0.50      1140
                 weighted avg       0.57      0.54      0.52      1140

[[ 

# SVM(RBF) on Red data

In [78]:
svm_rbf.fit(Ftrain,Ltrain)
y_pred = svm_rbf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.6464912280701754


In [79]:
#classification report
y_predict = svm_rbf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.71      0.66      0.68       124
                 early_blight       0.47      0.46      0.46       114
                      healthy       0.75      0.89      0.82       131
                  late_blight       0.59      0.57      0.58       124
                    leaf_mold       0.70      0.66      0.68       117
           septoria_leaf_spot       0.55      0.54      0.54       114
                  target_spot       0.60      0.68      0.64       111
          tomato_mosaic_virus       0.64      0.58      0.61        60
tomato_yellow_leaf_curl_virus       0.69      0.58      0.63       121
      two-spotted_spider_mite       0.71      0.77      0.74       124

                     accuracy                           0.65      1140
                    macro avg       0.64      0.64      0.64      1140
                 weighted avg       0.64      0.65      0.64      1140

[[ 

# SVM(sigmoid) on Red data

In [80]:
svm_si.fit(Ftrain,Ltrain)
y_pred = svm_si.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.30701754385964913


In [81]:
#classification report
y_predict = svm_si.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.38      0.42      0.40       124
                 early_blight       0.14      0.11      0.13       114
                      healthy       0.51      0.38      0.44       131
                  late_blight       0.19      0.20      0.20       124
                    leaf_mold       0.29      0.32      0.30       117
           septoria_leaf_spot       0.19      0.45      0.26       114
                  target_spot       0.37      0.31      0.33       111
          tomato_mosaic_virus       0.33      0.10      0.15        60
tomato_yellow_leaf_curl_virus       0.39      0.26      0.31       121
      two-spotted_spider_mite       0.54      0.41      0.47       124

                     accuracy                           0.31      1140
                    macro avg       0.33      0.30      0.30      1140
                 weighted avg       0.34      0.31      0.31      1140

[[5

# Random forest on Red channel data

In [82]:
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

0.4674270673393288

In [83]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.48      0.59      0.53       124
                 early_blight       0.37      0.12      0.18       114
                      healthy       0.49      0.87      0.63       131
                  late_blight       0.45      0.24      0.31       124
                    leaf_mold       0.54      0.57      0.56       117
           septoria_leaf_spot       0.53      0.46      0.49       114
                  target_spot       0.45      0.71      0.55       111
          tomato_mosaic_virus       0.00      0.00      0.00        60
tomato_yellow_leaf_curl_virus       0.40      0.31      0.35       121
      two-spotted_spider_mite       0.50      0.64      0.56       124

                     accuracy                           0.48      1140
                    macro avg       0.42      0.45      0.42      1140
                 weighted avg       0.44      0.48      0.44      1140

[[ 

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Gaussian Naive Bayes on Red channel data

In [84]:
gnb.fit(Ftrain, Ltrain)

GaussianNB()

In [85]:
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.4412280701754386

In [86]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.47      0.68      0.55       124
                 early_blight       0.30      0.26      0.28       114
                      healthy       0.52      0.74      0.61       131
                  late_blight       0.38      0.24      0.30       124
                    leaf_mold       0.59      0.38      0.46       117
           septoria_leaf_spot       0.38      0.45      0.41       114
                  target_spot       0.54      0.44      0.49       111
          tomato_mosaic_virus       0.40      0.58      0.47        60
tomato_yellow_leaf_curl_virus       0.31      0.12      0.18       121
      two-spotted_spider_mite       0.42      0.55      0.48       124

                     accuracy                           0.44      1140
                    macro avg       0.43      0.44      0.42      1140
                 weighted avg       0.43      0.44      0.42      1140

[[8

# Bernouli Naive bayes on Red data

In [87]:
bnb.fit(Ftrain, Ltrain)
y_pred = bnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.35526315789473684

In [88]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.38      0.69      0.49       124
                 early_blight       0.19      0.11      0.13       114
                      healthy       0.42      0.56      0.48       131
                  late_blight       0.22      0.11      0.15       124
                    leaf_mold       0.55      0.20      0.29       117
           septoria_leaf_spot       0.28      0.48      0.35       114
                  target_spot       0.42      0.37      0.39       111
          tomato_mosaic_virus       0.33      0.33      0.33        60
tomato_yellow_leaf_curl_virus       0.26      0.14      0.18       121
      two-spotted_spider_mite       0.42      0.52      0.46       124

                     accuracy                           0.36      1140
                    macro avg       0.35      0.35      0.33      1140
                 weighted avg       0.35      0.36      0.33      1140

[[8

# Decision Tree on Red channel

In [89]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.3587719298245614

In [90]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.38      0.34      0.36       124
                 early_blight       0.23      0.26      0.25       114
                      healthy       0.54      0.55      0.55       131
                  late_blight       0.35      0.28      0.31       124
                    leaf_mold       0.40      0.41      0.41       117
           septoria_leaf_spot       0.33      0.35      0.34       114
                  target_spot       0.29      0.32      0.31       111
          tomato_mosaic_virus       0.20      0.15      0.17        60
tomato_yellow_leaf_curl_virus       0.36      0.36      0.36       121
      two-spotted_spider_mite       0.44      0.48      0.46       124

                     accuracy                           0.36      1140
                    macro avg       0.35      0.35      0.35      1140
                 weighted avg       0.36      0.36      0.36      1140

[[4

# KNN on green channel data

In [91]:
#reading numpy array data
green_test = np.load('green_data.npy')
#reading csv file
green_df = pd.read_csv('green_data.csv')
loaded = []
for i in green_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
features = loaded
target = green_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
scaler = StandardScaler()
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.3570175438596491

In [92]:
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.25      0.75      0.38       124
                 early_blight       0.00      0.00      0.00       114
                      healthy       0.37      0.64      0.47       131
                  late_blight       0.28      0.19      0.22       124
                    leaf_mold       0.41      0.51      0.46       117
           septoria_leaf_spot       0.47      0.39      0.43       114
                  target_spot       0.47      0.19      0.27       111
          tomato_mosaic_virus       0.25      0.12      0.16        60
tomato_yellow_leaf_curl_virus       0.66      0.16      0.25       121
      two-spotted_spider_mite       0.48      0.44      0.46       124

                     accuracy                           0.36      1140
                    macro avg       0.36      0.34      0.31      1140
                 weighted avg       0.37      0.36      0.32      1140

[[9

# Logistic on Green channel data

In [93]:
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.32456140350877194

In [94]:
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.45      0.40      0.42       124
                 early_blight       0.29      0.32      0.31       114
                      healthy       0.44      0.41      0.43       131
                  late_blight       0.20      0.24      0.22       124
                    leaf_mold       0.41      0.44      0.43       117
           septoria_leaf_spot       0.28      0.28      0.28       114
                  target_spot       0.18      0.17      0.18       111
          tomato_mosaic_virus       0.23      0.17      0.19        60
tomato_yellow_leaf_curl_virus       0.32      0.29      0.30       121
      two-spotted_spider_mite       0.39      0.41      0.40       124

                     accuracy                           0.32      1140
                    macro avg       0.32      0.31      0.32      1140
                 weighted avg       0.33      0.32      0.32      1140

[[5

# SVM(linear) on green channel data

In [95]:
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.3675438596491228


In [96]:
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.35      0.48      0.40       124
                 early_blight       0.27      0.34      0.30       114
                      healthy       0.52      0.53      0.52       131
                  late_blight       0.25      0.26      0.25       124
                    leaf_mold       0.42      0.43      0.43       117
           septoria_leaf_spot       0.32      0.31      0.31       114
                  target_spot       0.23      0.22      0.22       111
          tomato_mosaic_virus       0.39      0.27      0.32        60
tomato_yellow_leaf_curl_virus       0.42      0.31      0.36       121
      two-spotted_spider_mite       0.57      0.46      0.51       124

                     accuracy                           0.37      1140
                    macro avg       0.37      0.36      0.36      1140
                 weighted avg       0.38      0.37      0.37      1140

[[5

# SVM(poly) on Green data

In [97]:
svm_poly.fit(Ftrain,Ltrain)
y_pred = svm_poly.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.48157894736842105


In [98]:
#classification report
y_predict = svm_poly.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.43      0.74      0.55       124
                 early_blight       0.54      0.25      0.34       114
                      healthy       0.49      0.73      0.59       131
                  late_blight       0.62      0.13      0.21       124
                    leaf_mold       0.34      0.74      0.46       117
           septoria_leaf_spot       0.65      0.46      0.54       114
                  target_spot       0.46      0.53      0.50       111
          tomato_mosaic_virus       0.29      0.03      0.06        60
tomato_yellow_leaf_curl_virus       0.57      0.39      0.46       121
      two-spotted_spider_mite       0.69      0.56      0.62       124

                     accuracy                           0.48      1140
                    macro avg       0.51      0.46      0.43      1140
                 weighted avg       0.52      0.48      0.45      1140

[[9

# SVM(RBF) on green data

In [99]:
svm_rbf.fit(Ftrain,Ltrain)
y_pred = svm_rbf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.6271929824561403


In [100]:
#classification report
y_predict = svm_rbf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.74      0.69      0.71       124
                 early_blight       0.44      0.46      0.45       114
                      healthy       0.69      0.83      0.76       131
                  late_blight       0.63      0.56      0.59       124
                    leaf_mold       0.66      0.63      0.65       117
           septoria_leaf_spot       0.58      0.67      0.62       114
                  target_spot       0.54      0.60      0.57       111
          tomato_mosaic_virus       0.57      0.43      0.49        60
tomato_yellow_leaf_curl_virus       0.65      0.56      0.60       121
      two-spotted_spider_mite       0.73      0.72      0.72       124

                     accuracy                           0.63      1140
                    macro avg       0.62      0.61      0.62      1140
                 weighted avg       0.63      0.63      0.63      1140

[[ 

# SVM(sigmoid) on green data

In [101]:
svm_si.fit(Ftrain,Ltrain)
y_pred = svm_si.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.28157894736842104


In [102]:
#classification report
y_predict = svm_si.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.36      0.41      0.38       124
                 early_blight       0.17      0.17      0.17       114
                      healthy       0.46      0.44      0.45       131
                  late_blight       0.20      0.27      0.23       124
                    leaf_mold       0.50      0.26      0.35       117
           septoria_leaf_spot       0.17      0.39      0.24       114
                  target_spot       0.20      0.21      0.20       111
          tomato_mosaic_virus       0.33      0.03      0.06        60
tomato_yellow_leaf_curl_virus       0.31      0.18      0.23       121
      two-spotted_spider_mite       0.45      0.31      0.36       124

                     accuracy                           0.28      1140
                    macro avg       0.32      0.27      0.27      1140
                 weighted avg       0.32      0.28      0.28      1140

[[5

# Random Forest on green channel data

In [103]:
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

0.4338670761131827

In [104]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.42      0.63      0.50       124
                 early_blight       0.37      0.19      0.25       114
                      healthy       0.43      0.70      0.54       131
                  late_blight       0.44      0.27      0.34       124
                    leaf_mold       0.47      0.51      0.49       117
           septoria_leaf_spot       0.53      0.46      0.50       114
                  target_spot       0.37      0.69      0.48       111
          tomato_mosaic_virus       0.00      0.00      0.00        60
tomato_yellow_leaf_curl_virus       0.61      0.33      0.43       121
      two-spotted_spider_mite       0.52      0.44      0.47       124

                     accuracy                           0.45      1140
                    macro avg       0.42      0.42      0.40      1140
                 weighted avg       0.44      0.45      0.42      1140

[[7

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Gaussian Naive bayes on Green channel data

In [105]:
gnb.fit(Ftrain, Ltrain)
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.3850877192982456

In [106]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.45      0.66      0.53       124
                 early_blight       0.37      0.40      0.38       114
                      healthy       0.42      0.63      0.51       131
                  late_blight       0.32      0.19      0.24       124
                    leaf_mold       0.56      0.23      0.33       117
           septoria_leaf_spot       0.36      0.45      0.40       114
                  target_spot       0.51      0.36      0.42       111
          tomato_mosaic_virus       0.26      0.52      0.34        60
tomato_yellow_leaf_curl_virus       0.19      0.08      0.11       121
      two-spotted_spider_mite       0.38      0.36      0.37       124

                     accuracy                           0.39      1140
                    macro avg       0.38      0.39      0.36      1140
                 weighted avg       0.39      0.39      0.37      1140

[[8

# Bernouli Naive bayes on green data

In [107]:
bnb.fit(Ftrain, Ltrain)
y_pred = bnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.32280701754385965

In [108]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.38      0.68      0.49       124
                 early_blight       0.26      0.18      0.21       114
                      healthy       0.39      0.53      0.45       131
                  late_blight       0.19      0.09      0.12       124
                    leaf_mold       0.59      0.22      0.32       117
           septoria_leaf_spot       0.31      0.56      0.40       114
                  target_spot       0.34      0.21      0.26       111
          tomato_mosaic_virus       0.15      0.30      0.20        60
tomato_yellow_leaf_curl_virus       0.05      0.02      0.03       121
      two-spotted_spider_mite       0.45      0.40      0.42       124

                     accuracy                           0.32      1140
                    macro avg       0.31      0.32      0.29      1140
                 weighted avg       0.32      0.32      0.30      1140

[[8

# Decision Tree on Green channel

In [109]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.33596491228070174

In [110]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.43      0.39      0.41       124
                 early_blight       0.24      0.24      0.24       114
                      healthy       0.52      0.51      0.52       131
                  late_blight       0.38      0.31      0.34       124
                    leaf_mold       0.33      0.38      0.36       117
           septoria_leaf_spot       0.31      0.39      0.35       114
                  target_spot       0.25      0.26      0.26       111
          tomato_mosaic_virus       0.27      0.25      0.26        60
tomato_yellow_leaf_curl_virus       0.27      0.27      0.27       121
      two-spotted_spider_mite       0.39      0.36      0.38       124

                     accuracy                           0.34      1140
                    macro avg       0.34      0.34      0.34      1140
                 weighted avg       0.35      0.34      0.34      1140

[[4

# KNN on blue channel data

In [111]:
#reading numpy array data
blue_test = np.load('blue_data.npy')
#reading csv file
blue_df = pd.read_csv('blue_data.csv')
loaded = []
for i in gray_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
features = loaded
target = blue_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.38859649122807016

In [112]:
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.28      0.74      0.40       124
                 early_blight       0.50      0.01      0.02       114
                      healthy       0.38      0.71      0.50       131
                  late_blight       0.31      0.23      0.26       124
                    leaf_mold       0.50      0.49      0.49       117
           septoria_leaf_spot       0.46      0.44      0.45       114
                  target_spot       0.47      0.24      0.32       111
          tomato_mosaic_virus       0.38      0.25      0.30        60
tomato_yellow_leaf_curl_virus       0.44      0.12      0.19       121
      two-spotted_spider_mite       0.55      0.52      0.53       124

                     accuracy                           0.39      1140
                    macro avg       0.43      0.38      0.35      1140
                 weighted avg       0.43      0.39      0.35      1140

[[9

# logistic on blue channel data

In [113]:
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.3140350877192982

In [114]:
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.48      0.41      0.44       124
                 early_blight       0.25      0.30      0.27       114
                      healthy       0.40      0.38      0.39       131
                  late_blight       0.18      0.19      0.19       124
                    leaf_mold       0.37      0.42      0.40       117
           septoria_leaf_spot       0.26      0.27      0.27       114
                  target_spot       0.20      0.18      0.19       111
          tomato_mosaic_virus       0.25      0.22      0.23        60
tomato_yellow_leaf_curl_virus       0.28      0.26      0.27       121
      two-spotted_spider_mite       0.41      0.44      0.43       124

                     accuracy                           0.31      1140
                    macro avg       0.31      0.31      0.31      1140
                 weighted avg       0.32      0.31      0.31      1140

[[5

# SVM(linear) on blue channel data

In [115]:
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.3570175438596491


In [116]:
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.33      0.44      0.38       124
                 early_blight       0.26      0.35      0.30       114
                      healthy       0.53      0.50      0.52       131
                  late_blight       0.23      0.22      0.22       124
                    leaf_mold       0.45      0.45      0.45       117
           septoria_leaf_spot       0.28      0.26      0.27       114
                  target_spot       0.28      0.26      0.27       111
          tomato_mosaic_virus       0.35      0.28      0.31        60
tomato_yellow_leaf_curl_virus       0.39      0.28      0.33       121
      two-spotted_spider_mite       0.52      0.45      0.48       124

                     accuracy                           0.36      1140
                    macro avg       0.36      0.35      0.35      1140
                 weighted avg       0.36      0.36      0.36      1140

[[5

# SVM(poly) on blue data

In [117]:
svm_poly.fit(Ftrain,Ltrain)
y_pred = svm_poly.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.5157894736842106


In [118]:
#classification report
y_predict = svm_poly.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.50      0.68      0.57       124
                 early_blight       0.55      0.21      0.30       114
                      healthy       0.50      0.92      0.65       131
                  late_blight       0.49      0.17      0.25       124
                    leaf_mold       0.47      0.63      0.54       117
           septoria_leaf_spot       0.68      0.44      0.53       114
                  target_spot       0.48      0.60      0.53       111
          tomato_mosaic_virus       0.60      0.05      0.09        60
tomato_yellow_leaf_curl_virus       0.44      0.56      0.49       121
      two-spotted_spider_mite       0.69      0.62      0.66       124

                     accuracy                           0.52      1140
                    macro avg       0.54      0.49      0.46      1140
                 weighted avg       0.53      0.52      0.48      1140

[[ 

# SVM(RBF) on blue data

In [119]:
svm_rbf.fit(Ftrain,Ltrain)
y_pred = svm_rbf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.6333333333333333


In [120]:
#classification report
y_predict = svm_rbf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.74      0.65      0.70       124
                 early_blight       0.46      0.49      0.47       114
                      healthy       0.70      0.86      0.77       131
                  late_blight       0.63      0.56      0.59       124
                    leaf_mold       0.69      0.65      0.67       117
           septoria_leaf_spot       0.58      0.62      0.60       114
                  target_spot       0.54      0.61      0.58       111
          tomato_mosaic_virus       0.57      0.47      0.51        60
tomato_yellow_leaf_curl_virus       0.65      0.60      0.63       121
      two-spotted_spider_mite       0.73      0.70      0.72       124

                     accuracy                           0.63      1140
                    macro avg       0.63      0.62      0.62      1140
                 weighted avg       0.64      0.63      0.63      1140

[[ 

# SVM(sigmoid) on blue data

In [121]:
svm_si.fit(Ftrain,Ltrain)
y_pred = svm_si.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.29210526315789476


In [122]:
#classification report
y_predict = svm_si.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.39      0.45      0.42       124
                 early_blight       0.19      0.20      0.20       114
                      healthy       0.43      0.39      0.41       131
                  late_blight       0.20      0.17      0.18       124
                    leaf_mold       0.56      0.27      0.37       117
           septoria_leaf_spot       0.18      0.39      0.24       114
                  target_spot       0.21      0.23      0.22       111
          tomato_mosaic_virus       0.43      0.05      0.09        60
tomato_yellow_leaf_curl_virus       0.31      0.32      0.31       121
      two-spotted_spider_mite       0.43      0.31      0.36       124

                     accuracy                           0.29      1140
                    macro avg       0.33      0.28      0.28      1140
                 weighted avg       0.33      0.29      0.29      1140

[[5

# Random Forest on blue channel data

In [123]:
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

0.4536082474226804

In [124]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.45      0.64      0.52       124
                 early_blight       0.38      0.14      0.21       114
                      healthy       0.46      0.80      0.59       131
                  late_blight       0.48      0.27      0.35       124
                    leaf_mold       0.56      0.58      0.57       117
           septoria_leaf_spot       0.55      0.47      0.51       114
                  target_spot       0.42      0.71      0.53       111
          tomato_mosaic_virus       0.00      0.00      0.00        60
tomato_yellow_leaf_curl_virus       0.52      0.35      0.42       121
      two-spotted_spider_mite       0.49      0.52      0.50       124

                     accuracy                           0.48      1140
                    macro avg       0.43      0.45      0.42      1140
                 weighted avg       0.45      0.48      0.44      1140

[[ 

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Gaussian Naive bayes on blue channel data

In [125]:
gnb.fit(Ftrain, Ltrain)
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.4087719298245614

In [126]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.47      0.69      0.56       124
                 early_blight       0.38      0.39      0.39       114
                      healthy       0.44      0.63      0.52       131
                  late_blight       0.34      0.20      0.25       124
                    leaf_mold       0.61      0.29      0.39       117
           septoria_leaf_spot       0.37      0.47      0.42       114
                  target_spot       0.50      0.38      0.43       111
          tomato_mosaic_virus       0.29      0.57      0.39        60
tomato_yellow_leaf_curl_virus       0.27      0.12      0.16       121
      two-spotted_spider_mite       0.39      0.40      0.40       124

                     accuracy                           0.41      1140
                    macro avg       0.41      0.41      0.39      1140
                 weighted avg       0.41      0.41      0.39      1140

[[8

# Bernouli Naive bayes on blue data

In [127]:
bnb.fit(Ftrain, Ltrain)
y_pred = bnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.3447368421052632

In [128]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.39      0.69      0.50       124
                 early_blight       0.28      0.18      0.22       114
                      healthy       0.40      0.54      0.46       131
                  late_blight       0.15      0.08      0.10       124
                    leaf_mold       0.63      0.31      0.41       117
           septoria_leaf_spot       0.31      0.53      0.39       114
                  target_spot       0.36      0.26      0.30       111
          tomato_mosaic_virus       0.18      0.27      0.22        60
tomato_yellow_leaf_curl_virus       0.17      0.09      0.12       121
      two-spotted_spider_mite       0.44      0.44      0.44       124

                     accuracy                           0.34      1140
                    macro avg       0.33      0.34      0.32      1140
                 weighted avg       0.34      0.34      0.32      1140

[[8

# Decision Tree on Blue channel

In [129]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.34649122807017546

In [130]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.40      0.42      0.41       124
                 early_blight       0.18      0.19      0.18       114
                      healthy       0.61      0.60      0.61       131
                  late_blight       0.33      0.32      0.33       124
                    leaf_mold       0.37      0.38      0.37       117
           septoria_leaf_spot       0.36      0.36      0.36       114
                  target_spot       0.30      0.31      0.30       111
          tomato_mosaic_virus       0.14      0.13      0.13        60
tomato_yellow_leaf_curl_virus       0.30      0.26      0.28       121
      two-spotted_spider_mite       0.27      0.27      0.27       124

                     accuracy                           0.34      1140
                    macro avg       0.32      0.32      0.32      1140
                 weighted avg       0.34      0.34      0.34      1140

[[5

In [131]:
end_time = time.time()
print(end_time - start_time)

755.3951661586761
